In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 34.9 MB/s 
     |████████████████████████████████| 7.6 MB 66.6 MB/s 
     |████████████████████████████████| 163 kB 72.3 MB/s 


In [3]:
cd /content/drive/MyDrive/commit_folder/competition/tour_ai/code

/content/drive/MyDrive/commit_folder/competition/tour_ai/code


In [4]:
server = 'colab'
from utils import dataset, models, utils

#
import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
if server == 'colab':
  mecab_data_path = '/content/drive/MyDrive/commit_folder/competition/tour_ai/data/mecab_data.csv'
  model_save_dict = '/content/drive/MyDrive/commit_folder/competition/tour_ai/data/model/bert_weight.pt'
  test_data_path = '/content/drive/MyDrive/commit_folder/competition/tour_ai/data/mecab_test_data.csv'
elif server == 'docker':
  mecab_data_path = '/tour_ai/data/mecab_data.csv'

In [6]:
data, label_encoding = utils.data_preprocess(mecab_data_path)
data = utils.tokenize_sentence(data)

# train_test_split
# train, valid = train_test_split(data, test_size=0.95, random_state=42)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [7]:
data_dataset = dataset.TextDataset(data)

# train_dataset = dataset.TextDataset(train)

# valid_dataset = dataset.TextDataset(valid)

In [8]:
model = models.TextModel(num_classes=len(label_encoding.classes_), text_model ='bert_klue')
model.load_state_dict(torch.load(model_save_dict))

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

<All keys matched successfully>

In [10]:
LR = 1e-6
EPOCHS = 5

In [11]:
utils.text_train(model, data_dataset, valid_data=None, lr=LR, epochs=EPOCHS, is_valid=False)

  0%|          | 0/1062 [00:00<?, ?it/s]/content/drive/MyDrive/commit_folder/competition/tour_ai/code/utils/models.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
100%|██████████| 1062/1062 [07:10<00:00,  2.46it/s]

Epochs : 5 | Train Loss : 0.2506 | Train Accuracy : 0.8593


In [12]:
torch.save(model.state_dict(), model_save_dict)

In [16]:
class TextDataset1(Dataset):
    def __init__(self, data, infer_yn=False):
        self.sentences = data['sentence'].values
        if not infer_yn:
            self.labels = data['label'].values
        self.infer_yn = infer_yn

    def __getitem__(self, index):
        # text data
        sentence = self.sentences[index]
        
        # label
        if self.infer_yn:
            return sentence
        else:
            label = self.labels[index]
            return sentence, label

    def __len__(self):
        return len(self.sentences)

In [17]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, Dataset
test = pd.read_csv(test_data_path)
test = test[['mecab_data']]
test.columns = ['sentence']
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-small')
test['sentence'] = test['sentence'].apply(lambda x: tokenizer(x, padding='max_length', max_length=300, truncation=True, return_tensors='pt'))

test_dataset = TextDataset1(test, infer_yn= True)
test_dataloader = DataLoader(test_dataset, batch_size = 8, shuffle=False)

def infer(model, test_dataloader):
  use_cuda = torch.cuda.is_available()
  device = torch.device('cuda' if use_cuda else 'cpu')

  if use_cuda:
    model = model.cuda()
  res = []
  for input in test_dataloader:
    mask = input['attention_mask'].squeeze(1).to(device)
    input_id = input['input_ids'].squeeze(1).to(device)
    output = model(input_id, mask)

    accuracy = output.argmax(dim=1)
    res.extend(accuracy.tolist())
  return res

result = infer(model, test_dataloader)

In [18]:
submission_path = '/content/drive/MyDrive/commit_folder/competition/tour_ai/data/sample_submission.csv'

In [19]:
submission = pd.read_csv(submission_path)
submission['cat3'] = result
submission['cat3'] = label_encoding.inverse_transform(submission['cat3'])
submission.to_csv(submission_path, index=False)